In this example we are going to scrape board actions from the [Texas Medical Board](https://profile.tmb.state.tx.us/SearchBA.aspx). Unlike last time, we're doing to be downloading a bunch of PDFs with details!

Traditionally Python programmers use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) to scrape content from the interent. Instead of being *traditional*, we're going to use [Playwright](https://playwright.dev/python/), a **browser automation tool**! This means you actually control the browser! Filling out forms, clicking buttons, downloading documents... it's magic!!!✨✨✨

# Texas Medical Board Actions Details

- Filling out text inputs
- Inspecting the page
- Combining dataframes
- Looping through licenses
- Downloading PDFs (using Firefox)

## Installation

We need to install a few tools first! Remove the `#` and run the cell to install the Python packages and browsers that we'll need for our scraping adventure.

In [ ]:
# %pip install --quiet lxml html5lib beautifulsoup4 pandas
# %pip install --quiet playwright
# !playwright install

## Opening up the browser and visiting our destination

We've been using Chromium (basically Chrome) for most of our exercises, but in this case we're using Firefox! Chromium for some reason sometimes gets blocked, while Firefox doesn't. Not sure why!

In [ ]:
from playwright.async_api import async_playwright

# "Hey, open up a browser"
playwright = await async_playwright().start()
browser = await playwright.firefox.launch(headless=False)

# Create a new browser window
page = await browser.new_page()

In [ ]:
await page.goto("https://profile.tmb.state.tx.us/SearchBA.aspx?eb2d4a70-6591-4ad4-ae6d-a2727e84cb39")

# Filling in a single license and searching

Filling in text fields, clicking, waiting for buttons to show up and clicking. Nothing crazy!

In [ ]:
# M6992
# Q5611
# M1444
await page.locator("#BodyContent_tbLicense").fill("M6992")

In [ ]:
await page.get_by_role("button", name="Search").click()

In [ ]:
await page.get_by_text("Document").click()

### Opening up the documents section

In [ ]:
await page.get_by_role("button", name="ExpandWeb Documents").wait_for()

In [ ]:
await page.get_by_role("button", name="ExpandWeb Documents").click()

I dug into the code on the page and we have 3 links. 

In [ ]:
links = page.locator(".doclink")
await links.count()

### Downloading one of the PDFs

By default this opens up in Chrome/Chromium as an in-page PDF. We change to Firefox to make it actually download.

In [ ]:
# Start waiting for the download
async with page.expect_download() as download_info:
    # Perform the action that initiates download
    await links.nth(1).click()
download = await download_info.value

print("Saving as", download.suggested_filename)

# Wait for the download process to complete and save the downloaded file somewhere
await download.save_as(download.suggested_filename)

### Return to the previous page

In [ ]:
#await page.get_by_text("Back").click()
await page.get_by_role("link", name="Back").click()

## Putting it all together

If we had a CSV with licenses in it, we can read in the CSV file and extract a list of licenses.

In [ ]:
import pandas as pd

df = pd.read_csv("licenses.csv")
df.head()

In [ ]:
licenses = df['License'].unique().tolist()

# We don't like the NONE one, so we'll remove it.
licenses.remove('NONE')
licenses

I don't think we have very many licenses in the CSV, so we'll try it again with a list of licenses that I put together manually.

In [ ]:
licenses = ['M6992', 'Q5611', 'M1444']

for license in licenses:
    print("Searching for", license)
    await page.goto("https://profile.tmb.state.tx.us/SearchBA.aspx?eb2d4a70-6591-4ad4-ae6d-a2727e84cb39")
    
    # Fill out license
    await page.locator("#BodyContent_tbLicense").fill(license)
    
    # Click search button
    await page.get_by_role("button", name="Search").click()
    
    # Move into Documents section
    await page.get_by_text("Document").click()
    
    # Expand documents tab
    await page.get_by_role("button", name="ExpandWeb Documents").wait_for()
    await page.get_by_role("button", name="ExpandWeb Documents").click()

    links = page.locator(".doclink")
    count = await links.count()
    count

    # Download all documents
    for i in range(count):
        # Wait for download
        async with page.expect_download() as download_info:
            await links.nth(i).click()
        download = await download_info.value
        
        print("Saving as", download.suggested_filename)
        
        # Wait for the download process to complete and save the downloaded file somewhere
        await download.save_as(download.suggested_filename)